![h2o](https://i.imgur.com/wi7SoU9.jpg)

In [1]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split

import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.3" 2019-04-16; OpenJDK Runtime Environment (build 11.0.3+7); OpenJDK 64-Bit Server VM (build 11.0.3+7, mixed mode)
  Starting server from /home/jagadeesh/anaconda3/envs/ml/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplv98pr_8
  JVM stdout: /tmp/tmplv98pr_8/h2o_jagadeesh_started_from_python.out
  JVM stderr: /tmp/tmplv98pr_8/h2o_jagadeesh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_jagadeesh_0ypm0l
H2O cluster total nodes:,1
H2O cluster free memory:,1.932 Gb
H2O cluster total cores:,6
H2O cluster allowed cores:,6
H2O cluster status:,"accepting new members, healthy"


In [3]:
#read dataset

data = pd.read_csv("/home/jagadeesh/Datasets/diabetes.csv")

In [4]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
#split data into train and test.

train,test = train_test_split(data,test_size=0.10)

In [6]:
#convert pd DF to h2o DF.

train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
x = train_h2o.columns
y = "Outcome"  #target
x.remove(y)

In [8]:
# For binary classification, response should be a factor
train_h2o[y] = train_h2o[y].asfactor()
test_h2o[y] = test_h2o[y].asfactor()

In [9]:
AutoML = H2OAutoML(max_models=5)
AutoML.train(x, y, training_frame=train_h2o)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [10]:
lb = AutoML.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
GLM_grid_1_AutoML_20190614_233018_model_1,0.83703,0.473108,0.229766,0.390774,0.152704
StackedEnsemble_BestOfFamily_AutoML_20190614_233018,0.836519,0.475175,0.233184,0.394178,0.155376
StackedEnsemble_AllModels_AutoML_20190614_233018,0.836037,0.47595,0.234752,0.394568,0.155684
XGBoost_2_AutoML_20190614_233018,0.829911,0.484112,0.235786,0.398583,0.158868
XGBoost_1_AutoML_20190614_233018,0.829572,0.478557,0.230369,0.397947,0.158362
XGBoost_3_AutoML_20190614_233018,0.826363,0.485346,0.237386,0.399719,0.159775
DRF_1_AutoML_20190614_233018,0.795936,0.786436,0.271852,0.414167,0.171534


In [11]:
y_pred = AutoML.leader.predict(test_h2o)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [12]:
y_pred["predict"]

predict
1
0
0
0
0
1
1
0
0
0
